In [ ]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz, process
import re
import json

# Sample DataFrame setup
data = {
    'Ticket': range(1, 21),
    'Business Service': [
        'Microsoft Outlook', 'Microsoft Windows', 'Laptop', 'Python',
        'voice log', 'crm service', 'cwhh vdi', 'vantage agent portal',
        'Microsoft Excel', 'MS Excel', 'Excel 365', 'outlook email',
        'Outlook client', 'Windows 10', 'Microsoft Windows 11',
        'laptop hardware', 'laptop support', 'python development',
        'Python scripting', 'CRM system'
    ]
}
df = pd.DataFrame(data)

# Method 1: LLM-based approach
def llm_based_standardization(df, chat_function):
    """
    Use LLM to group similar services and standardize them
    """
    # Get unique services and their counts
    service_counts = df['Business Service'].value_counts()
    unique_services = service_counts.index.tolist()
    
    # Create prompt for LLM to group similar services
    prompt = f"""
    I have a list of business services that need to be standardized. Please group similar services together and for each group, suggest the most representative name (preferably the one that appears most frequently).

    Services with their counts:
    {dict(service_counts)}

    Please return a JSON object where:
    - Keys are the standardized service names
    - Values are lists of all variations that should map to that standardized name

    Example format:
    {{
        "Microsoft Excel": ["Microsoft Excel", "MS Excel", "Excel 365"],
        "Microsoft Outlook": ["Microsoft Outlook", "outlook email", "Outlook client"]
    }}

    Focus on grouping services that refer to the same underlying technology or service.
    """
    
    try:
        # Call the LLM function
        response = chat_function(prompt)
        
        # Parse the JSON response
        # Note: You might need to extract JSON from the response text
        standardization_map = json.loads(response)
        
        # Create a mapping from original to standardized
        service_mapping = {}
        for standard_name, variations in standardization_map.items():
            for variation in variations:
                service_mapping[variation] = standard_name
        
        # Apply mapping to create new column
        df['New Services'] = df['Business Service'].map(
            lambda x: service_mapping.get(x, x)
        )
        
        return df, service_mapping
        
    except Exception as e:
        print(f"Error in LLM processing: {e}")
        return df, {}

# Method 2: Traditional NLP approach
def traditional_nlp_standardization(df, similarity_threshold=80):
    """
    Use fuzzy matching and pattern recognition to standardize services
    """
    # Get service counts
    service_counts = df['Business Service'].value_counts()
    unique_services = list(service_counts.index)
    
    # Preprocessing function
    def preprocess_service(service):
        # Convert to lowercase, remove extra spaces, common prefixes/suffixes
        service = service.lower().strip()
        service = re.sub(r'\bmicrosoft\b|\bms\b', 'microsoft', service)
        service = re.sub(r'\b365\b|\boffice\b', '', service).strip()
        service = re.sub(r'\s+', ' ', service)
        return service
    
    # Create groups of similar services
    groups = []
    used_services = set()
    
    for service in unique_services:
        if service in used_services:
            continue
            
        # Find similar services
        similar_services = [service]
        used_services.add(service)
        
        for other_service in unique_services:
            if other_service in used_services:
                continue
                
            # Check similarity using multiple methods
            similarity_scores = [
                fuzz.ratio(preprocess_service(service), preprocess_service(other_service)),
                fuzz.partial_ratio(service.lower(), other_service.lower()),
                fuzz.token_sort_ratio(service.lower(), other_service.lower()),
                fuzz.token_set_ratio(service.lower(), other_service.lower())
            ]
            
            max_similarity = max(similarity_scores)
            
            # Also check for keyword overlap
            service_words = set(preprocess_service(service).split())
            other_words = set(preprocess_service(other_service).split())
            word_overlap = len(service_words & other_words) / max(len(service_words), len(other_words))
            
            if max_similarity >= similarity_threshold or word_overlap >= 0.6:
                similar_services.append(other_service)
                used_services.add(other_service)
        
        if len(similar_services) > 1:
            groups.append(similar_services)
    
    # For each group, find the service with highest count
    service_mapping = {}
    for group in groups:
        # Get counts for services in this group
        group_counts = {service: service_counts[service] for service in group}
        # Find the service with maximum count
        standard_service = max(group_counts, key=group_counts.get)
        
        # Map all services in group to the standard one
        for service in group:
            service_mapping[service] = standard_service
    
    # Apply mapping
    df['New Services'] = df['Business Service'].map(
        lambda x: service_mapping.get(x, x)
    )
    
    return df, service_mapping, groups

# Method 3: Hybrid approach
def hybrid_standardization(df, chat_function, similarity_threshold=75):
    """
    Combine traditional NLP for initial grouping and LLM for validation/refinement
    """
    # Step 1: Use traditional NLP for initial grouping
    df_temp, initial_mapping, groups = traditional_nlp_standardization(
        df.copy(), similarity_threshold
    )
    
    # Step 2: Use LLM to validate and refine the groups
    if groups:
        prompt = f"""
        I've used fuzzy matching to group similar business services. Please review these groups and suggest improvements:

        Groups found:
        {json.dumps(groups, indent=2)}

        Service counts:
        {dict(df['Business Service'].value_counts())}

        Please return a JSON object with refined groupings where:
        - Keys are the best representative names for each service category
        - Values are lists of all variations that should map to that name
        
        Validate that the groupings make sense semantically.
        """
        
        try:
            response = chat_function(prompt)
            refined_mapping = json.loads(response)
            
            # Create final mapping
            final_service_mapping = {}
            for standard_name, variations in refined_mapping.items():
                for variation in variations:
                    final_service_mapping[variation] = standard_name
            
            df['New Services'] = df['Business Service'].map(
                lambda x: final_service_mapping.get(x, x)
            )
            
            return df, final_service_mapping
            
        except Exception as e:
            print(f"Error in LLM refinement: {e}")
            return df_temp, initial_mapping
    
    return df_temp, initial_mapping

# Example usage and comparison
def compare_methods(df):
    """
    Compare the effectiveness of different methods
    """
    print("Original services:")
    print(df['Business Service'].value_counts())
    print("\n" + "="*50 + "\n")
    
    # Traditional NLP method
    df_traditional, mapping_traditional, groups = traditional_nlp_standardization(df.copy())
    print("Traditional NLP Results:")
    print("Groups found:", groups)
    print("Standardized services:")
    print(df_traditional['New Services'].value_counts())
    print(f"Reduced from {df['Business Service'].nunique()} to {df_traditional['New Services'].nunique()} unique services")
    
    return df_traditional, mapping_traditional

# Mock chat function for testing (replace with your actual LLM endpoint)
def mock_chat_function(prompt):
    """
    Mock LLM function - replace this with your actual chat function
    """
    # This is a mock response - your actual LLM should analyze the prompt
    mock_response = '''
    {
        "Microsoft Excel": ["Microsoft Excel", "MS Excel", "Excel 365"],
        "Microsoft Outlook": ["Microsoft Outlook", "outlook email", "Outlook client"],
        "Microsoft Windows": ["Microsoft Windows", "Windows 10", "Microsoft Windows 11"],
        "Laptop": ["Laptop", "laptop hardware", "laptop support"],
        "Python": ["Python", "python development", "Python scripting"],
        "CRM Service": ["crm service", "CRM system"],
        "Voice Log": ["voice log"],
        "CWHH VDI": ["cwhh vdi"],
        "Vantage Agent Portal": ["vantage agent portal"]
    }
    '''
    return mock_response

# Run comparison
if __name__ == "__main__":
    # Test traditional method
    df_result, mapping = compare_methods(df)
    
    print("\n" + "="*50 + "\n")
    print("Final DataFrame with standardized services:")
    print(df_result[['Business Service', 'New Services']].head(10))
    
    # Test LLM method (uncomment when you have actual chat function)
    # df_llm, mapping_llm = llm_based_standardization(df.copy(), your_chat_function)
    # print("LLM Results:", df_llm['New Services'].value_counts())